In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs

### Web Scraping #1: Getting a list of GPUs from the first page of newegg.ca

In [6]:
# this is getting the source code
newegg_url = requests.get('https://www.newegg.ca/p/pl?d=graphics+card')
# turn it into a beatifulsoup object - lxml is the parser
soup = bs.BeautifulSoup(newegg_url.text, 'lxml')
# grab each product 
containers = soup.findAll('div', {'class':'item-container'})
price = soup.findAll('li', {'class':'price-current'})
shipping = soup.findAll('li', {'class':'price-ship'})

brand = []
name = []
price_list = []
shipping_list = []

# getting names
for container in containers:
    if container.div.div.a.img != None:
        brand.append(container.div.div.a.img['title'])
        name.append(container.a.img['title'])

# getting prices
for item in price:
    price_list.append(item.strong.text + item.sup.text)

# getting prices of shipping
for item in shipping:
    shipping_list.append(item.text.split()[0])

print('Printing 5 items of each list')
print(shipping_list[:5])
print(price_list[:5])
print(brand[:5])
print(name[:5])

Printing 5 items of each list
[]
[]
['GIGABYTE', 'MSI', 'ASUS', 'EVGA', 'MSI']
36


In [ ]:
# since I want to sort items by price, I need to get rid of commas and convert them into floats 
price_list = [float(x.replace(',','')) for x in price_list]

In [ ]:
frame = {'Name':name, 'Brand':brand, 'Price':price_list, 'Shipping':shipping_list}
gpu_df = pd.DataFrame(frame)
gpu_df.sort_values(by='Price', ascending=False, inplace=True)
gpu_df.head(10)

These are the 10 most expensive GPUs from the first page

In [7]:
steam_url = requests.get('https://store.steampowered.com/search/?specials={}')
soup = bs.BeautifulSoup(steam_url.text, 'lxml')
all_container = soup.findAll('div', {'class':'responsive_search_name_combined'})

game_list = []
discount_list = []
original_price = []
dis_price_list = []

for container in all_container:
    game = container.findChildren()[1].text
    game_list.append(game)
    
    # percentage of discount 
    discount = container.findChildren('div')[3]
    if discount.div.span == None:
        discount_list.append('No Discount')
    else:
        discount_list.append(discount.div.span.text.replace('-',''))
    
    # original price
    o_price = container.findChildren('div')[3].findChildren('div')[1]
    if o_price.strike == None:
        original_price.append('0')
    else:
        original_price.append(o_price.strike.text.split()[1])
    
    # discounted price
    dis_price_line = container.findChildren('div')[3].findChildren('div')[1]
    line_split = dis_price_line.text.strip().split()
    if not line_split:
        dis_price_list.append('0')
    else:
        dis_price_list.append(line_split[-1])

# checking if the number of items match
print(len(dis_price_list))
print(len(original_price))
print(len(discount_list))
print(len(game_list))

50
50
50
50


In [8]:
original_price = np.array([float(x) for x in original_price])
dis_price_list = np.array([float(x) for x in dis_price_list])

frame = {'Game':game_list, 'Original Price':original_price, 'Discounted Amount':original_price-dis_price_list,
         'Discounted Price':dis_price_list, 'Discount %':discount_list}
steam_df = pd.DataFrame(frame)
steam_df.sort_values(by='Discounted Amount', inplace=True, ascending=False)
steam_df.reset_index(drop=True, inplace=True)
steam_df.head(20)

,Game,Original Price,Discounted Amount,Discounted Price,Discount %
0,Darksiders Blades & Whip Franchise Pack,166.46,131.64,34.82,79%
1,Arkane 20th Anniversary Bundle,170.44,127.47,42.97,75%
2,Wolfenstein Alt History Collection,144.96,103.60,41.36,71%
3,DOOM Franchise Bundle,190.45,97.03,93.42,51%
4,Dishonored: Complete Collection,109.99,77.00,32.99,70%
5,Elder Scrolls Summer Bundle,94.97,68.00,26.97,72%
6,RAGE 2,79.99,64.00,15.99,80%
7,The Elder Scrolls V: Skyrim VR,79.99,56.00,23.99,70%
8,Fallout 4: Game of the Year Edition,79.99,56.00,23.99,70%
9,Darksiders III,79.99,53.60,26.39,67%
